In [94]:
import numpy as np
import geopandas as gpd
import rasterio
from pathlib import Path
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling

In [95]:
class GeoSpatialDataProcessor:
    """
    A class to process and visualize geospatial data from a local GeoPackage file.
    """

    def __init__(self, geopackage_path: str, layer_name: str, output_dir: str, code_column: str):
        """
        Initialize the GeoSpatialDataProcessor with the GeoPackage details and output directory.
        """
        self.geopackage_path = Path(geopackage_path)
        self.layer_name = layer_name
        self.output_dir = Path(output_dir)
        self.code_column = code_column

        if not self.geopackage_path.exists():
            raise FileNotFoundError(f"GeoPackage path {geopackage_path} does not exist.")
        self.output_dir.mkdir(parents=True, exist_ok=True)

        self.land_use_code_mapping = {
            1: [211, 212, 213, 221, 222, 223, 241, 242, 243, 244],  # agriculture
            2: [111, 112],  # urban
            3: [121, 122, 123, 124, 131, 132, 133],  # industry
            4: [523],  # water_bodies
            5: [311, 312, 313],  # nature_forest
            6: [231],  # meadow
            7: [141, 142, 321, 322, 324, 331, 411, 421, 423, 511, 512]  # other
        }

        self.category_names = {
            1: 'agriculture',
            2: 'urban',
            3: 'industry',
            4: 'water_bodies',
            5: 'nature_forest',
            6: 'meadow',
            7: 'other'
        }

    def read_data(self) -> gpd.GeoDataFrame:
        """
        Reads the data from the GeoPackage file.
        """
        try:
            data = gpd.read_file(self.geopackage_path, layer=self.layer_name)
            if data.empty:
                raise ValueError("The data is empty.")
            print("First few rows of the data:")
            print(data.head())
            return data
        except Exception as e:
            print(f"An error occurred while reading the data: {e}")
            raise

    def reclassify_data(self, data: gpd.GeoDataFrame) -> tuple:
        """
        Reclassifies the raster data into broader land use categories.
        """
        print("Reclassifying data")

        # Convert land use codes in the data to integers
        data[self.code_column] = data[self.code_column].astype(int)

        # Print unique land use codes in the data
        unique_codes = data[self.code_column].unique()
        print(f"Unique land use codes in the data: {unique_codes}")

        # Create a mapping from the original land use codes to the new categories
        reverse_mapping = {code: category for category, codes in self.land_use_code_mapping.items() for code in codes}

        # Create a new column for the reclassified codes
        data['reclassified'] = data[self.code_column].map(reverse_mapping)

        # Handle missing or unexpected codes
        missing_mask = data['reclassified'].isna()
        if missing_mask.any():
            print(f"{missing_mask.sum()} rows have missing or unexpected codes.")
            data.loc[missing_mask, 'reclassified'] = 0  # Assign a new category for unknown codes

        data['reclassified'] = data['reclassified'].astype(int)

        # Update category names to include the 'unknown' category
        self.category_names[0] = 'unknown'

        # Display the unique categories after reclassification
        unique, counts = np.unique(data['reclassified'], return_counts=True)
        print(f"Unique categories after reclassification: {dict(zip(unique, counts))}")

        return data['reclassified'], self.category_names

    def export_data(self, reclassified: gpd.GeoDataFrame, data: gpd.GeoDataFrame, category: int):
        """
        Exports the data of each land use type as a separate tiff file.
        """
        rasterized = None  # Initialize rasterized variable
        try:
            output_path = self.output_dir / f"{self.category_names[category]}.tif"
            if output_path.exists():
                print(f"File {output_path} already exists, skipping export.")
                return

            # Only include the geometries that match the current category
            shapes = [(geom, 1) for geom, value in zip(data.geometry, reclassified) if value == category]

            # Print CRS of the data
            print(f"CRS of the data: {data.crs}")

            # Define the desired resolution
            resolution = 30  # meters

            # Calculate the width and height of the output raster
            width = int((data.total_bounds[2] - data.total_bounds[0]) / resolution)
            height = int((data.total_bounds[3] - data.total_bounds[1]) / resolution)

            # Define the rasterization parameters
            transform, width, height = calculate_default_transform(data.crs, 'EPSG:3857', width, height, *data.total_bounds, resolution=(resolution, resolution))
            out_shape = (height, width)

            # Perform rasterization
            rasterized = rasterio.features.rasterize(shapes, out_shape=out_shape, transform=transform, fill=-9999, default_value=1, dtype=rasterio.float32)

            # Ensure the rasterized array is not empty
            if not np.any(rasterized):
                print(f"No data for category {category}. Skipping export.")
                return

            # Define the metadata for the output raster
            meta = {
                'driver': 'GTiff',
                'height': rasterized.shape[0],
                'width': rasterized.shape[1],
                'count': 1,
                'dtype': rasterio.float32,
                'crs': 'EPSG:3857',
                'transform': transform,
                'nodata': -9999
            }

            # Write the rasterized array to a GeoTIFF file
            with rasterio.open(output_path, 'w', **meta) as dst:
                dst.write(rasterized, 1)

            print(f"Successfully exported data to {output_path}")
        except Exception as e:
            print(f"An error occurred while exporting data: {e}")
            raise

    def process_data(self):
        """
        Main method to process the data.
        """
        data = self.read_data()
        if data is not None:
            reclassified, category_names = self.reclassify_data(data)
            categories = [category for category in np.unique(reclassified) if category in category_names]
            print(f"Processing categories: {categories}")

            # Run export_data method directly without multiprocessing
            for category in categories:
                print(f"\nProcessing category {category}: {self.category_names[category]}")
                self.export_data(reclassified, data, category)

In [96]:
if __name__ == "__main__":
    processor = GeoSpatialDataProcessor(
        geopackage_path='corine_data_landcover/zuid-holland/Results/U2018_CLC2018_V2020_20u1.gpkg',
        layer_name='U2018_CLC2018_V2020_20u1',
        output_dir='corine_reclassify_GTiff',
        code_column='Code_18'
    )
    processor.process_data()

First few rows of the data:
   OBJECTID Code_18 Remark     Area_Ha  Shape_Length    Shape_Area  \
0   1570275     112   None  159.046461   6514.551351  1.590465e+06   
1   1570318     112   None   37.401350   3300.250382  3.740135e+05   
2   1570324     112   None   62.038357   3990.132619  6.203836e+05   
3   1570325     112   None   52.610017   3237.477859  5.261002e+05   
4   1570351     112   None  174.041492   6552.864183  1.740415e+06   

                                            geometry  
0  POLYGON ((468651.184 6745659.977, 468786.427 6...  
1  POLYGON ((477542.762 6749182.121, 477509.903 6...  
2  POLYGON ((455634.865 6747345.528, 455315.104 6...  
3  POLYGON ((464041.012 6749066.793, 464221.592 6...  
4  POLYGON ((494883.960 6753496.606, 494864.230 6...  
Reclassifying data
Unique land use codes in the data: [112 121 122 123 124 132 133 141 142 211 222 231 242 243 311 312 313 321
 322 324 331 411 421 423 511 512 523]
Unique categories after reclassification: {1: 339, 2: 47